# Testing your app

Once your app is running you can query it. You can simply do that by navigating to the URL that ngrok gave you above or through code in a different python session. For example, once the frontend app is running, you can navigate to:
http://YOUR_SERVER_DOMAIN/search?query=hello+world where YOUR_SERVER_DOMAIN is something like XXXX-XX-XX-XX-XX.ngrok.io, which is printed above in Colab or that is your external IP on GCP.

The code below shows how to issue a query from python. This is also how our testing code will issue queries to your search engine, so make sure to test your search engine this way after you deploy it to GCP and before submission. Command line instructions for deploying your search engine to GCP are available at `run_frontend_in_gcp.sh`. Note that we will not only issue training queries to your search engine, but also test queries, i.e. queries that you've never seen before.

In [2]:
import json

with open('queries_train.json', 'rt') as f:
  queries = json.load(f)

In [3]:
def average_precision(true_list, predicted_list, k=40):
    true_set = frozenset(true_list)
    predicted_list = predicted_list[:k]
    precisions = []
    for i,doc_id in enumerate(predicted_list):        
        if doc_id in true_set:
            prec = (len(precisions)+1) / (i+1)            
            precisions.append(prec)
    if len(precisions) == 0:
        return 0.0
    sum = 0
    for item in precisions:
      sum += item
    return round(item/len(precisions),3)

In [4]:
import requests
from time import time
# url = 'http://35.232.59.3:8080'
# place the domain you got from ngrok or GCP IP below. 
url = 'http://2589-34-86-111-80.ngrok.io'

qs_res = []
for q, true_wids in queries.items():
  duration, ap = None, None
  t_start = time()
  try:
    res = requests.get(url + '/search_body', {'query': q}, timeout=35)
    duration = time() - t_start
    if res.status_code == 200:
      pred_wids, _ = zip(*res.json())
      ap = average_precision(true_wids, pred_wids)
  except:
    pass
  
  qs_res.append((q, duration, ap))
print(qs_res)
sum = 0
for tup in qs_res:
  sum += tup[2]
avg = sum/len(qs_res)
print(f"average precision is {avg}")

[('best marvel movie', 3.1101293563842773, 0.026), ('How do kids come to world?', 2.8349804878234863, 0.0), ('Information retrieval', 0.8622043132781982, 0.026), ('LinkedIn', 0.06853628158569336, 0.031), ('How to make coffee?', 0.8889825344085693, 0.027), ('Ritalin', 0.05195283889770508, 0.025), ('How to make wine at home?', 2.8573379516601562, 0.062), ('Most expensive city in the world', 3.4178199768066406, 1.0), ('India', 1.047957181930542, 0.025), ('how to make money fast?', 1.659315824508667, 0.026), ('Netflix', 0.14011859893798828, 0.143), ('Apple computer', 0.9464163780212402, 0.091), ('The Simpsons', 0.08690023422241211, 0.071), ('World cup', 2.049920082092285, 0.077), ('How to lose weight?', 0.8704202175140381, 0.028), ('Java', 0.17435407638549805, 0.059), ('Air Jordan', 1.806544303894043, 0.034), ('how to deal with depression?', 0.5784780979156494, 0.025), ('How do you make gold', 1.1889348030090332, 0.026), ('Marijuana', 0.09549951553344727, 0.028), ('How to make hummus', 0.6